This notebook extracts the resting-state temporal dynamics of brain networks.


We first use an atlas to construct a reasonably realistic model of interacting regions or nodes. We then use the atlas to mask BOLD activities of the nodes, remove confounds from the data, and finally extract the time-series for each node. This process ends up with a matrix of time-series of size n_subjects * n_nodes * n_timesteps, e.g., 32*39*125 in case of using regions of interest from \cite{@dosenbach2007}.